In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Define file path
file_path = r'C:\Users\dharpure.1\Downloads\Discharge paper\M3.xlsx'

# Define file save path
path_save = r'C:\Users\dharpure.1\Downloads\Discharge paper\Model 2\M3\\'
if not os.path.exists(path_save):
    os.makedirs(path_save)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

def normalize(df):
    # Normalize data from scratch
    normalized_df = pd.DataFrame()
    min_max_value = {}
    for column in df.columns:
        if column != 'Date':
            col_min = df[column].min()
            col_max = df[column].max()
            normalized_df[column] = ((df[column] - col_min) / (col_max - col_min)) * 2 - 1
        else:
            normalized_df[column] = df[column]
        if column == 'Discharge': 
            min_max_value[column] = {'min': col_min, 'max': col_max}
    
    return normalized_df, min_max_value

def de_normalize(discharge, min_max_value):
    discharge_min = min_max_value['Discharge']['min']
    discharge_max = min_max_value['Discharge']['max']
    denormalized_df = (discharge + 1) / 2 * (discharge_max - discharge_min) + discharge_min
    return np.round(denormalized_df, 2)

def calculate_matrix(observed, predicted):
    RMSE = np.sqrt(mean_squared_error(observed, predicted))
    mean_observed = np.mean(observed)
    NSE = 1 - (np.sum((predicted - observed)**2) / np.sum((observed - mean_observed)**2))
    r = pearsonr(observed, predicted)
    return {'RMSE': round(RMSE, 3), 'NSE': round(NSE, 3), 'R': round(r[0], 3)}

# Read the Excel file into a pandas DataFrame
df_data = pd.read_excel(file_path)
print(df_data)

In [ ]:
model_number = int(input ('Please setect input scenario data using number, i.e., M1 indicates 1, M2 indicates 2 and so on'))

import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mpl

def cross_corr_figure(df_data):
    index_to_stop = df_data.index[df_data['Discharge'].isna()].tolist()[0]
    model_df = df_data.loc[:index_to_stop-1].dropna()
    df =  model_df.iloc[:, 1:]
    c = df.shape[1]
    title_data = ['(a) Ta (\N{DEGREE SIGN}C)', '(b) Precipitation (mm)', '(c) Snowfall (mm)', '(d) ET (mm)', '(e) SCA (%)', '(f) Discharge (m\u00b3/s)']
    
    # Global figure properties
    fontsize = 8
    legend_fontsize = 9
    linewidth = 0.3
    markersize = 3
    markeredgewidth = 0.5
    dpi = 300

    params = {
        'axes.labelsize': fontsize,
        'axes.titlesize': fontsize,
        'font.size': fontsize,
        'figure.titlesize': fontsize,
        'legend.fontsize': legend_fontsize,
        'xtick.labelsize': fontsize,
        'ytick.labelsize': fontsize,
        'axes.linewidth': linewidth,
        'savefig.dpi': dpi,
        'font.family': 'sans-serif',
        'figure.facecolor': 'w',
    }

    mpl.rcParams.update(params)
    ccf_data = {}
    slice_ccf = 11
    for col in range(c):
        discharge = df.iloc[:, -1]
        var = df.iloc[:, col]
        ccf_data[title_data[col]] = sm.tsa.ccf(discharge, var)[:slice_ccf]
    x = [i for i in range(slice_ccf)]
    fig, axs = plt.subplots(nrows=int(c/2), ncols=2, figsize=(10, 8), constrained_layout=True)
    for ax, title, var_ccf in zip(axs.flat, list(ccf_data.keys()), list(ccf_data.values())):
        ax.set_title(title)
        ax.plot(x, var_ccf, 'o', color="r", ls='-', ms=4, markevery=None)
        ax.set_xlabel("Lag time (days)")
        ax.set_ylabel("Cross_corr")
    
    return ccf_data

def build_lagged_features(s, lag=2, dropna=True, col_name='None'):
    if type(s) is pd.Series:
        the_range = range(lag+1)
        res = pd.concat([s.shift(i+1) for i in the_range], axis=1)
        res.columns = [f'{col_name}_{i}' for i in the_range]
    else:
        print('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res

data = list(cross_corr_figure(df_data).values())
df =  df_data.iloc[:, 1:]

# Please select max_corr_index as per the scenario dataset
# Index represents [Ta, Pt, Snowfall, Et, SCA, discharge]
if(model_number==1):
    max_corr_index = [2, 1, 1, 0]  # for M1
elif(model_number==2):
    max_corr_index = [2, 1, 1, 0, 0]  # for M2
elif(model_number==3):
    max_corr_index = [2, 1, 1, 0, 1]  # for M3   
elif(model_number==4):
    max_corr_index = [2, 1, 1, 0, 0,1]  # for M4
    
new_dict = {}
for corr, col_name in zip(max_corr_index, df.columns.to_list()):
    res = build_lagged_features(df[col_name], lag=corr, dropna=False, col_name=col_name)
    new_dict[col_name] = res
    
df_features = pd.DataFrame()
df_features['Date'] = df_data.iloc[:, 0]
for key_var in new_dict.keys():
    for key_lag in new_dict[key_var].keys():
        df_features[key_lag] = new_dict[key_var][key_lag]
df_features['Discharge'] = df_data.iloc[:, -1]

col_list = list(df_data.columns[1:])
dict_lag_corr = {}
for row in range(np.array(data).shape[0]):
    dict_lag_corr[col_list[row]] = np.array(data)[row, :]
corr_df = pd.DataFrame(dict_lag_corr)

In [ ]:
df_features

## Data prepration

In [ ]:
index_to_stop = df_features.index[df_features['Discharge'].isna()].tolist()[0]

# Select rows up to the index_to_stop
model_df = df_features.loc[:index_to_stop-1].dropna()

# Normalize the model data
norm_model, min_max_value = normalize(model_df)

# Split data into train and test sets
train_data = norm_model[norm_model['Date'] <= '2010-09-30']
test_data = norm_model[norm_model['Date'] >= '2010-10-01']

# Prepare data for prediction
predict_df = df_features.iloc[index_to_stop:, :-1]
norm_predict, _ = normalize(predict_df)

# Separate features and target for training
X_train = train_data.iloc[:, 1:-1]  # Features
y_train = train_data.iloc[:, -1]    # Target
y_train_denor = de_normalize(y_train, min_max_value)

# Separate features and target for testing
X_test = test_data.iloc[:, 1:-1]
y_test = de_normalize(test_data.iloc[:, -1], min_max_value)

In [ ]:
X_train

## Random Forest

In [ ]:
# Create a Random Forest model
model = RandomForestRegressor(n_estimators=500, random_state=42, max_features='log2')

# Train the model
model.fit(X_train, y_train)

# Make predictions on the train set
y_pred_train = de_normalize(model.predict(X_train), min_max_value)

# Make predictions on the test set
y_pred_test = de_normalize(model.predict(X_test), min_max_value)

# Define the path for saving results
path = path_save + 'RandomForest\\'
if not os.path.exists(path):
    os.makedirs(path)

# Save train data results
train_result = {
    'Date': pd.to_datetime(train_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_train_denor,
    'Predicted': y_pred_train
}
df_train_result = pd.DataFrame(train_result)
df_train_result.to_excel(path + 'train_result_rf.xlsx', index=False)
print("Train results saved in Excel")

# Save test data results
test_result = {
    'Date': pd.to_datetime(test_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_test,
    'Predicted': y_pred_test
}
df_test_result = pd.DataFrame(test_result)
df_test_result.to_excel(path + '\\Test_result_rf.xlsx', index=False)
print("Test results saved in Excel")

# Calculate and save metrics
columns = ['RMSE', 'NSE', 'R']
matrics_rf = pd.DataFrame(columns=columns)
train_df_rf = pd.DataFrame([calculate_matrix(y_train_denor, y_pred_train)], index=['Train'])
test_df_rf = pd.DataFrame([calculate_matrix(y_test, y_pred_test)], index=['Test'])
matrics_rf = pd.concat([train_df_rf, test_df_rf], ignore_index=False)
matrics_rf.to_excel(path + 'Matrics_rf.xlsx')
print("Metrics results saved in Excel")

# Save feature importance
dict_rf = {
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}
dict_df_rf = pd.DataFrame(dict_rf)
dict_df_rf.to_excel(path + 'Feature_importance_rf.xlsx')
print("Feature importance results saved in Excel")

In [ ]:
matrics_rf

## Support Vector Machine

In [ ]:
# Create a Support Vector Machine (SVM) model
model = SVR(kernel='rbf', C=10, epsilon=0.1)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the train set
y_pred_train = de_normalize(model.predict(X_train), min_max_value)

# Make predictions on the test set
y_pred_test = de_normalize(model.predict(X_test), min_max_value)

# Define the path for saving results
path = path_save + 'SVM\\'
if not os.path.exists(path):
    os.makedirs(path)

# Save train data results
train_result = {
    'Date': pd.to_datetime(train_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_train_denor,
    'Predicted': y_pred_train
}
df_train_result = pd.DataFrame(train_result)
df_train_result.to_excel(path + 'train_result_svm.xlsx', index=False)
print("Train results saved in Excel")

# Save test data results
test_result = {
    'Date': pd.to_datetime(test_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_test,
    'Predicted': y_pred_test
}
df_test_result = pd.DataFrame(test_result)
df_test_result.to_excel(path + '\\Test_result_svm.xlsx', index=False)
print("Test results saved in Excel")

# Calculate and save metrics
columns = ['RMSE', 'NSE', 'R']
matrics_svm = pd.DataFrame(columns=columns)
train_df_svm = pd.DataFrame([calculate_matrix(y_train_denor, y_pred_train)], index=['Train'])
test_df_svm = pd.DataFrame([calculate_matrix(y_test, y_pred_test)], index=['Test'])
matrics_svm = pd.concat([train_df_svm, test_df_svm], ignore_index=False)
matrics_svm.to_excel(path + 'Matrics_svm.xlsx')
print("Metrics results saved in Excel")

In [ ]:
matrics_svm

## ANN

In [ ]:
# Your data and model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.dropout1 = nn.Dropout(p=0.10)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(p=0.10)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.dropout1(out)
        out = F.relu(self.fc2(out))
        out = self.dropout2(out)
        out = self.fc3(out)
        return out

model = NeuralNetwork(X_train.shape[1], 1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0025)

# Training loop
num_epochs = 250
train_loss = []

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze().float()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on test set
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor).cpu().numpy()

# Convert predictions back to original scale if needed
y_pred_test_denor = de_normalize(torch.tensor(y_pred_test), min_max_value)

# Save results to Excel
path = path_save + 'ANN\\'
if not os.path.exists(path):
    os.makedirs(path)

# Test data save
test_result = {
    'Date': pd.to_datetime(test_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_test,
    'Predicted': y_pred_test_denor.flatten()
}
df_test_result = pd.DataFrame(test_result)
df_test_result.to_excel(os.path.join(path, 'Test_result_ANN.xlsx'), index=False)
print("Test results saved in Excel")

# Predictions on training set
with torch.no_grad():
    y_pred_train = model(X_train_tensor).cpu().numpy()

# Convert predictions back to original scale if needed
y_pred_train_denor = de_normalize(y_pred_train, min_max_value)

# Save results to Excel
train_result = {
    'Date': pd.to_datetime(train_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_train_denor,
    'Predicted': y_pred_train_denor.flatten()
}
df_train_result = pd.DataFrame(train_result)
df_train_result.to_excel(os.path.join(path, 'train_result_ANN.xlsx'), index=False)
print("Train results saved in Excel")

# Metrics calculation and save
train_metrics = calculate_matrix(y_train_denor, y_pred_train_denor.flatten())
test_metrics = calculate_matrix(y_test, y_pred_test_denor.numpy().flatten())

matrics_ANN = pd.DataFrame([train_metrics, test_metrics], index=['Train', 'Test'])
matrics_ANN.to_excel(os.path.join(path, 'Matrics_ANN.xlsx'))
print("Metrics results saved in Excel")

torch.save(model, path+str(model_number)+'.pth')


In [ ]:
matrics_ANN

## Bidirectional LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os

# Function to prepare sequences for LSTM
def series_to_supervised_gap(sequences, n_steps_in=1, n_steps_out=1):
    X = list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        seq_x = sequences[i:end_ix]
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix > len(sequences):
            break
        X.append(seq_x)
    return np.array(X)

# Your data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to tensors and move to device
X_train_tensor = torch.tensor(series_to_supervised_gap(X_train, n_steps_in=time_step), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(series_to_supervised_gap(X_test, n_steps_in=time_step), dtype=torch.float32).to(device)

# Define the Bidirectional LSTM model
class BidirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rates):
        super(BidirectionalLSTM, self).__init__()
        self.lstm_layers = nn.ModuleList([
            nn.LSTM(input_size if i == 0 else hidden_sizes[i - 1] * 2,  # Adjusted input size for bidirectional
                    hidden_size,
                    num_layers=2,
                    dropout=dropout_rates[i],
                    batch_first=True,
                    bidirectional=True)
            for i, (hidden_size, dropout_rate) in enumerate(zip(hidden_sizes, dropout_rates))
        ])
        self.linear = nn.Linear(hidden_sizes[-1] * 2, output_size)

    def forward(self, x):
        for lstm_layer in self.lstm_layers:
            x, _ = lstm_layer(x)
        out = self.linear(x[:, -1, :])
        return out

# Initialize model parameters and optimizer
input_size = X_train_tensor.shape[-1]
output_size = 1
hidden_sizes = [128, 64]
dropout_rates = [0.1, 0.1]

model = BidirectionalLSTM(input_size, hidden_sizes, output_size, dropout_rates).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# Training loop
nepochs = 250
train_loss = []

for epoch in range(nepochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.flatten(), y_train_tensor)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
    print(f'Epoch [{epoch+1}/{nepochs}], Loss: {loss.item():.4f}')

# Evaluation on the test set
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor).cpu().numpy()

# Convert predictions back to the original scale if needed
y_pred_test_denor = de_normalize(torch.tensor(y_pred_test.flatten()), min_max_value)

# Save results to Excel
path = path_save + 'BiLSTM\\'
os.makedirs(path, exist_ok=True)

# Test data save
test_result = {
    'Date': pd.to_datetime(test_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_test,
    'Predicted': y_pred_test_denor.numpy()
}
df_test_result = pd.DataFrame(test_result)
df_test_result.to_excel(os.path.join(path, 'Test_result_LSTM_Bidirectional.xlsx'), index=False)
print("Test results saved in Excel")

# Make predictions on the training set
with torch.no_grad():
    y_pred_train = model(X_train_tensor).cpu().numpy()

# Convert predictions back to the original scale if needed
y_pred_train_dnor = de_normalize(torch.tensor(y_pred_train.flatten()), min_max_value)

# Train data save
train_result = {
    'Date': pd.to_datetime(train_data['Date']).dt.strftime('%Y-%m-%d'),
    'Observed': y_train_denor,
    'Predicted': y_pred_train_dnor.numpy()
}
df_train_result = pd.DataFrame(train_result)
df_train_result.to_excel(os.path.join(path, 'train_result_LSTM_Bidirectional.xlsx'), index=False)
print("Train results saved in Excel")

# Metrics save
train_metrics = calculate_matrix(y_train_denor, y_pred_train_dnor.numpy())
test_metrics = calculate_matrix(y_test, y_pred_test_denor.numpy())

matrics_LSTM = pd.DataFrame([train_metrics, test_metrics], index=['Train', 'Test'])
matrics_LSTM.to_excel(os.path.join(path, 'Matrics_LSTM_Bidirectional.xlsx'))
print("Metrics results saved in Excel")

torch.save(model, path+str(model_number)+'.pth')

In [ ]:
matrics_LSTM